In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq langchain --progress-bar off
!pip install -qqq chromadb --progress-bar off
!pip install -qqq sentence-transformers --progress-bar off
!pip install -qqq auto-gptq --progress-bar off
!pip install -qqq einops --progress-bar off
!pip install -qqq unstructured --progress-bar off
!pip install -qqq transformers --progress-bar off
!pip install -qqq torch --progress-bar off
!pip install -qqq langchain-community --progress-bar off
!pip install -qqq gradio --progress-bar off


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unstructured-client 0.31.3 requires aiofiles>=24.1.0, but you have aiofiles 23.2.1 which is incompatible.


In [ ]:
from pathlib import Path

import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, GenerationConfig, TextStreamer, pipeline

/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:410: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:418: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float16)


## Data

In [ ]:
questions_dir = Path("skyscanner")
questions_dir.mkdir(exist_ok=True, parents=True)


def write_file(question, answer, file_path):
    text = f"""
Q: {question}
A: {answer}
""".strip()
    with Path(questions_dir / file_path).open("w") as text_file:
        text_file.write(text)

In [ ]:
write_file(
    question="Sexual abuse",
    answer="""
The Protection of Children from Sexual Offences (POCSO) Act, 2012, is an Indian law that protects
children from sexual abuse and exploitation. The Act defines various sexual oAenses against children,
including penetrative and non-penetrative sexual assault, sexual harassment, and children's use for
pornographic purposes. The Act provides stringent punishment for those guilty of committing these
oAences, including imprisonment and fines. It also places a significant responsibility on the police to
protect children during the investigative process. The Act requires police personnel to make immediate
arrangements for the care and protection of the child in case of a report of sexual abuse. This includes
obtaining emergency medical treatment and placing the child in a shelter home if necessary. The Act also
mandates the police to inform the Child Welfare Committee within 24 hours of receiving the report to
ensure further arrangements for the safety and security of the child.""".strip(),
    file_path="question_1.txt",
)

In [ ]:
write_file(
    question="Child marriage",
    answer="""Laws related to child marriage in India prohibit such unions until the minimum age of 21 for males and 18
for females is reached, applying uniformly across all religions. The Prohibition of Child Marriage Act,
2006, strictly forbids child marriage and addresses related matters. If you are being forced to marry, visit
the nearest police station to file a report and reach out to a trusted adult.""".strip(),
    file_path="question_2.txt",
)

In [ ]:
write_file(
    question="Forced to work, employment, labour",
    answer="""The Child Labour (Prohibition and Regulation) Act 1986 prohibits the employment of children under 14 in
certain hazardous occupations and processes. It regulates the working conditions of children in other
occupations. It provides a penalty for employers who violate the Act's provisions and empowers the
government to inspect any premises to enforce the Act's provisions. It has been amended several times
to include more hazardous occupations in the list and to increase the penalties for violating the Act's
provisions.""".strip(),
    file_path="question_3.txt",
)

In [ ]:
write_file(
    question="I am a child and I committed a crime",
    answer="""The Juvenile Justice (Care and Protection of Children) Act, 2015 is an Indian law that provides for the care,
protection, and rehabilitation of children in conflict with the law or in need of care and protection. The Act
replaces the Juvenile Justice (Care and Protection of Children) Act, 2000, and provides a child-friendly
approach to administering justice for children. The Act provides for establishing Juvenile Justice Boards
and Child Welfare Committees to deal with cases involving children in conflict with the law or needing
care and protection. It also provides for establishing special homes, observation homes, and after-care
organizations to provide care and protection to children. """.strip(),
    file_path="question_4.txt",
)

In [ ]:
write_file(
    question="Adoption",
    answer="""The Juvenile Justice (Care and Protection of Children) Act, 2015 lays down guidelines and procedures for
the adoption of children, with the Central Adoption Resource Authority (CARA) serving as the nodal body
for the same. The Act also aims to ensure that the best interests of the child are upheld throughout the
adoption process. It is an important step towards protecting and rehabilitating children in India. The
eAective implementation of the Act is essential to ensure that children are treated with dignity and
allowed to lead productive life.
""".strip(),
    file_path="question_5.txt",
)

In [ ]:
write_file(
    question="Education, not being allowed to study",
    answer="""The Right of Children to Free and Compulsory Education (RTE) Act, 2009 is an Indian law that guarantees
the right of every child to free and compulsory education till the age of 14. This provides free and
compulsory education to all children between the ages of 6 and 14 years and mandates the state to
ensure the availability of and access to quality education for all children in the country. This Act
recognizes that education is a fundamental right and aims to reduce the drop-out rate and improve the
quality of education in India. Under the RTE Act, the government must provide school infrastructure and
facilities, including classrooms, toilets, and drinking water facilities. The Act also mandates the
appointment of trained teachers and the provision of pedagogical resources.""".strip(),
    file_path="question_6.txt",
)

In [ ]:
write_file(
    question="Child trafficking, kidnapping",
    answer="""The Immoral TraAic (Prevention) Act 1956, as amended in 1986, is an Indian law that criminalizes the
procurement of minors for prostitution and the operation of brothels. The Act provides punishment for
those found guilty of these crimes and includes provisions for protecting and caring for rescued children.
""".strip(),
    file_path="question_7.txt",
)

In [ ]:
write_file(
    question="Abandonment, Improper care and neglect",
    answer="""Section 317 states that whoever being the father or mother of a child under the age of twelve years, or having the care of such
child, shall expose or leave such child in any place with the intention of wholly abandoning such child,
shall be punished with imprisonment of either description for a term which may extend to seven years, or
with fine, or with both. If you are experiencing neglect from your parents or your guardian, please contact
the child helpline 1098 for assistance and care.
""".strip(),
    file_path="question_8.txt",
)

In [ ]:
write_file(
    question="Bullying, Cyberbullying",
    answer="""Currently, there is no specific law in India that holds students legally responsible for bullying.
However, incidents of bullying can be reported under the broader framework of student
harassment laws and child protection legislation, which can lead to punitive measures against
the perpetrators. Parents and students can report incidents to the school authorities, the child
welfare committees, or the police. Suppose the educational institution fails to take appropriate
action. In that case, the matter can be escalated to higher authorities, such as the District
Education Officer or the State Commission for Protection of Child Rights. """.strip(),
    file_path="question_9.txt",
)

In [ ]:
write_file(
    question="Child Safety Guidelines, how to stay safe?",
    answer="""Remember the name, address and telephone number of your parents or guardian. Carry the
details around using a keychain. Do not accept gifts, especially something you can eat from strangers. Speak to a trusted adult such as your parent or teacher if you feel uncomfortable about
something. Seek help from the police by dialling 100 if you think something illegal is going on, you can find information about illegal activities and crimes on this chatbot. Do not open the door directly when someone knocks. Never follow a stranger if they ask you to come with them.
Do not share your details on any social media applications, gaming applications as it is severely detrimental to your privacy.
""".strip(),
    file_path="question_10.txt",
)

In [ ]:
write_file(
    question="What can your website do",
    answer="""Aims for child safety and empowerment and includes functions like:
1. A chatbot to educate children on child protection laws
2. A portal to contact lawyers
3. Legal Document Summarization system
4. A registration module that allows lawyers to register themselves on the website""".strip(),
    file_path="question_11.txt",
)

In [ ]:
write_file(
    question="How to get help?",
    answer="""Helplines include Swadhar NGO – 93910112852, Women and Children Helpline – 181, Childline - 1098, Police - 100. All are available 24 hours throughout the day""".strip(),
    file_path="question_12.txt",
)

## Model

In [ ]:
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
MODEL_NAME = "TheBloke/Nous-Hermes-13B-GPTQ"
model = AutoGPTQForCausalLM.from_quantized(
    MODEL_NAME, use_safetensors=True, trust_remote_code=True, device=DEVICE
)
generation_config = GenerationConfig.from_pretrained(model_name_or_path)

WARNING - Exllamav2 kernel is not installed, reset disable_exllamav2 to True. This may because you installed auto_gptq using a pre-build wheel on Windows, in which exllama_kernels are not compiled. To use exllama_kernels to further speedup inference, you can re-install auto_gptq from source.
WARNING - CUDA kernels for auto_gptq are not installed, this will result in very slow inference speed. This may because:
1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.
1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.


model.safetensors:   0%|          | 0.00/7.45G [00:00<?, ?B/s]

INFO - The layer lm_head is not quantized.
INFO:auto_gptq.modeling._base:The layer lm_head is not quantized.


generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

In [ ]:
question = (
    "Which programming language is more suitable for a beginner: Python or JavaScript?"
)
prompt = f"""
### Instruction: {question}
### Response:
""".strip()

In [ ]:
%%time
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(DEVICE)
with torch.inference_mode():
    output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=512)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


CPU times: user 19.8 s, sys: 11.5 s, total: 31.4 s
Wall time: 32.3 s


In [ ]:
print(tokenizer.decode(output[0]))

<s> ### Instruction: Which programming language is more suitable for a beginner: Python or JavaScript?
### Response:Python is generally considered more suitable for beginners due to its readability and simplicity compared to JavaScript.</s>


In [ ]:
generation_config

GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "pad_token_id": 0
}

In [ ]:
streamer = TextStreamer(
    tokenizer, skip_prompt=True, skip_special_tokens=True, use_multiprocessing=False
)

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=2048,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
    generation_config=generation_config,
    streamer=streamer,
    batch_size=1,
)

Device set to use cuda:0
The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'Jam

In [ ]:
llm = HuggingFacePipeline(pipeline=pipe)

<ipython-input-26-b97a330b5f24>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [ ]:
response = llm(prompt)

<ipython-input-27-a8a9d1e67c96>:1: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(prompt)
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `

Python is generally considered to be more suitable for beginners due to its readability and simplicity compared to JavaScript.


## Embed Documents

In [ ]:
embeddings = HuggingFaceEmbeddings()

<ipython-input-28-d0c9174021d8>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
<ipython-input-28-d0c9174021d8>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
loader = DirectoryLoader("./skyscanner/", glob="**/*txt")
documents = loader.load()
len(documents)

12

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [ ]:
texts[4]

Document(metadata={'source': 'skyscanner/question_10.txt'}, page_content='Q: Child Safety Guidelines how to stay safe? A: Remember the name address and telephone number of your parents or guardian. Carry the details around using a keychain. Do not accept gifts especially something you can eat from strangers. Speak to a trusted adult such as your parent or teacher if you feel uncomfortable about something. Seek help from the police by dialling 100 if you think something illegal is going on you can find information about illegal activities and crimes on this chatbot. Do not open the door directly when someone knocks. Never follow a stranger if they ask you to come with them. Do not share your details on any social media applications gaming applications as it is severely detrimental to your privacy.')

In [ ]:
db = Chroma.from_documents(texts, embeddings)

In [ ]:
db.similarity_search("child trafficking")

[Document(metadata={'source': 'skyscanner/question_7.txt'}, page_content='Q: Child trafficking, kidnapping\n\nA: The Immoral TraAic (Prevention) Act 1956, as amended in 1986, is an Indian law that criminalizes the\n\nprocurement of minors for prostitution and the operation of brothels. The Act provides punishment for\n\nthose found guilty of these crimes and includes provisions for protecting and caring for rescued children.'),
 Document(metadata={'source': 'skyscanner/question_1.txt'}, page_content="Q: Sexual abuse\n\nA: The Protection of Children from Sexual Offences (POCSO) Act, 2012, is an Indian law that protects\n\nchildren from sexual abuse and exploitation. The Act defines various sexual oAenses against children,\n\nincluding penetrative and non-penetrative sexual assault, sexual harassment, and children's use for\n\npornographic purposes. The Act provides stringent punishment for those guilty of committing these"),
 Document(metadata={'source': 'skyscanner/question_5.txt'}, pa

## Conversational Chain

In [ ]:
template = """
### Instruction: You're an agent that is giving information about child protection laws. Use only the chat history and the following information
{context}
to answer in a helpful manner to the question. If you don't know the answer - say that you don't know.
Keep your replies short, compassionate and informative.
{chat_history}
### Input: {question}
### Response:
""".strip()

In [ ]:
prompt = PromptTemplate(
    input_variables=["context", "question", "chat_history"], template=template
)

In [ ]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    human_prefix="### Input",
    ai_prefix="### Response",
    output_key="answer",
    return_messages=True,
)

<ipython-input-36-fd38385448f6>:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [ ]:
chain = ConversationalRetrievalChain.from_llm(
    llm,
    chain_type="stuff",
    retriever=db.as_retriever(),
    memory=memory,
    combine_docs_chain_kwargs={"prompt": prompt},
    return_source_documents=True,
    verbose=True,
)

In [ ]:
question = "I am being forced to work"
answer = chain(question)

<ipython-input-38-4349663c96ce>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = chain(question)
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
### Instruction: You're an agent that is giving information about child protection laws. Use only the chat history and the following information
Q: Forced to work, employment, labour

A: The Child Labour (Prohibition and Regulation) Act 1986 prohibits the employment of children under 14 in

certain hazardous occupations and processes. It regulates the working conditions of children in other

occupations. It provides a penalty for employers who violate the Act's provisions and empowers the

government to inspect any premises to enforce the Act's provisions. It has been amended several times

Q: Education, not being allowed to study

A: The Right of Children to Free and Compulsory Education (RTE) Act, 2009 is an Indian law that guarantees

the right of every child to free and compulsory education till the age of 14. This provides free and

compulsory education to all children between

In [ ]:
answer.keys()

dict_keys(['question', 'chat_history', 'answer', 'source_documents'])

In [ ]:
answer["source_documents"]

[Document(metadata={'source': 'skyscanner/question_3.txt'}, page_content="Q: Forced to work, employment, labour\n\nA: The Child Labour (Prohibition and Regulation) Act 1986 prohibits the employment of children under 14 in\n\ncertain hazardous occupations and processes. It regulates the working conditions of children in other\n\noccupations. It provides a penalty for employers who violate the Act's provisions and empowers the\n\ngovernment to inspect any premises to enforce the Act's provisions. It has been amended several times"),
 Document(metadata={'source': 'skyscanner/question_6.txt'}, page_content='Q: Education, not being allowed to study\n\nA: The Right of Children to Free and Compulsory Education (RTE) Act, 2009 is an Indian law that guarantees\n\nthe right of every child to free and compulsory education till the age of 14. This provides free and\n\ncompulsory education to all children between the ages of 6 and 14 years and mandates the state to\n\nensure the availability of and

In [ ]:
question = "How can I get help"
response = chain(question)



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: I am being forced to work
Assistant: ### Instruction: You're an agent that is giving information about child protection laws. Use only the chat history and the following information
Q: Forced to work, employment, labour

A: The Child Labour (Prohibition and Regulation) Act 1986 prohibits the employment of children under 14 in

certain hazardous occupations and processes. It regulates the working conditions of children in other

occupations. It provides a penalty for employers who violate the Act's provisions and empowers the

government to inspect any premises to enforce the Act's provisions. It has been amended several times

Q: Education, not being allowed to study

A: The Right of Children to Free and Compulsory Education (RTE) Act, 2009 is an Indian law tha

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Can you provide me with information on how to seek assistance if I am being forced to work?

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
### Instruction: You're an agent that is giving information about child protection laws. Use only the chat history and the following information
Q: Child trafficking, kidnapping

A: The Immoral TraAic (Prevention) Act 1956, as amended in 1986, is an Indian law that criminalizes the

procurement of minors for prostitution and the operation of brothels. The Act provides punishment for

those found guilty of these crimes and includes provisions for protecting and caring for rescued children.

Q: Forced to work, employment, labour

A: The Child Labour (Prohibition and Regulation) Act 1986 prohibits the employment of children under 14 in

certain hazardous occupations and processes. It regulates the working conditions of children in other

occupations. It provides a penalty for

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


If you are being forced to work against your will, you can reach out to local authorities such as the police or child welfare services for help. Additionally, you can call the National Child Labor Helpline at 18001113700 or visit their website at www.childlineindia.org.in for support and guidance.

> Finished chain.

> Finished chain.


## QA Chain with Memory

In [ ]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    human_prefix="### Input",
    ai_prefix="### Response",
    input_key="question",
    output_key="output_text",
    return_messages=False,
)

chain = load_qa_chain(
    llm, chain_type="stuff", prompt=prompt, memory=memory, verbose=True
)

<ipython-input-49-803022ac6874>:10: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain = load_qa_chain(


In [ ]:
question = "I was sexually abused"
docs = db.similarity_search(question)
answer = chain.run({"input_documents": docs, "question": question})

<ipython-input-50-22e080772bb6>:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = chain.run({"input_documents": docs, "question": question})
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
### Instruction: You're an agent that is giving information about child protection laws. Use only the chat history and the following information
Q: Sexual abuse

A: The Protection of Children from Sexual Offences (POCSO) Act, 2012, is an Indian law that protects

children from sexual abuse and exploitation. The Act defines various sexual oAenses against children,

including penetrative and non-penetrative sexual assault, sexual harassment, and children's use for

pornographic purposes. The Act provides stringent punishment for those guilty of committing these

Q: Child trafficking, kidnapping

A: The Immoral TraAic (Prevention) Act 1956, as amended in 1986, is an Indian law that criminalizes the

procurement of minors for prostitution and the operation of brothels. The Act provides punishment for

those found guilty of these crimes and includes provisions for protecting and caring 

In [ ]:
question = "How can i be safe"
docs = db.similarity_search(question)
answer = chain.run({"input_documents": docs, "question": question})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
### Instruction: You're an agent that is giving information about child protection laws. Use only the chat history and the following information
Q: Child Safety Guidelines how to stay safe? A: Remember the name address and telephone number of your parents or guardian. Carry the details around using a keychain. Do not accept gifts especially something you can eat from strangers. Speak to a trusted adult such as your parent or teacher if you feel uncomfortable about something. Seek help from the police by dialling 100 if you think something illegal is going on you can find information about illegal activities and crimes on this chatbot. Do not open the door directly when someone knocks. Never follow a stranger if they ask you to come with them. Do not share your details on any social media applications gaming applications as it is severely detrimental to your privacy.

Q: How to get 

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


To stay safe, remember the name, address, and telephone number of your parents or guardian. Carry their details around using a keychain. Don't accept gifts, especially something you can eat, from strangers. Speak to a trusted adult such as your parent or teacher if you feel uncomfortable about something. Seek help from the police by dialing 100 if you think something illegal is going on. Don't open the door directly when someone knocks. Never follow a stranger who asks you to come with them. Don't share your details on any social media applications or gaming applications as it is severely detrimental to your privacy.

> Finished chain.

> Finished chain.


In [ ]:
print(answer.strip())

### Instruction: You're an agent that is giving information about child protection laws. Use only the chat history and the following information
Q: Child Safety Guidelines how to stay safe? A: Remember the name address and telephone number of your parents or guardian. Carry the details around using a keychain. Do not accept gifts especially something you can eat from strangers. Speak to a trusted adult such as your parent or teacher if you feel uncomfortable about something. Seek help from the police by dialling 100 if you think something illegal is going on you can find information about illegal activities and crimes on this chatbot. Do not open the door directly when someone knocks. Never follow a stranger if they ask you to come with them. Do not share your details on any social media applications gaming applications as it is severely detrimental to your privacy.

Q: How to get help?

A: Helplines include Swadhar NGO – 93910112852, Women and Children Helpline – 181, Childline - 1098

## Support Chatbot

In [ ]:
DEFAULT_TEMPLATE = """
### Instruction: You're an agent that is giving information about child protection laws. Use only the chat history and the following information
{context}
to answer in a helpful manner to the question. If you don't know the answer - say that you don't know.
Keep your replies short, compassionate and informative.
{chat_history}
### Input: {question}
### Response:
""".strip()


class Chatbot:
    def __init__(
        self,
        text_pipeline: HuggingFacePipeline,
        embeddings: HuggingFaceEmbeddings,
        documents_dir: Path,
        prompt_template: str = DEFAULT_TEMPLATE,
        verbose: bool = False,
    ):
        prompt = PromptTemplate(
            input_variables=["context", "question", "chat_history"],
            template=prompt_template,
        )
        self.chain = self._create_chain(text_pipeline, prompt, verbose)
        self.db = self._embed_data(documents_dir, embeddings)

    def _create_chain(
        self,
        text_pipeline: HuggingFacePipeline,
        prompt: PromptTemplate,
        verbose: bool = False,
    ):
        memory = ConversationBufferMemory(
            memory_key="chat_history",
            human_prefix="### Input",
            ai_prefix="### Response",
            input_key="question",
            output_key="output_text",
            return_messages=False,
        )

        return load_qa_chain(
            text_pipeline,
            chain_type="stuff",
            prompt=prompt,
            memory=memory,
            verbose=verbose,
        )

    def _embed_data(
        self, documents_dir: Path, embeddings: HuggingFaceEmbeddings
    ) -> Chroma:
        loader = DirectoryLoader(documents_dir, glob="**/*txt")
        documents = loader.load()
        text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=0)
        texts = text_splitter.split_documents(documents)
        return Chroma.from_documents(texts, embeddings)

    def __call__(self, user_input: str) -> str:
        docs = self.db.similarity_search(user_input)
        return self.chain.run({"input_documents": docs, "question": user_input})

In [ ]:
chatbot = Chatbot(llm, embeddings, "./skyscanner/")

In [ ]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

while True:
    user_input = input("You: ")
    if user_input.lower() in ["bye", "goodbye"]:
        break
    answer = chatbot(user_input)
    print()

You: I was sexually abused
### Instruction: You're an agent that is giving information about child protection laws. Use only the chat history and the following information
Q: Sexual abuse

A: The Protection of Children from Sexual Offences (POCSO) Act, 2012, is an Indian law that protects

children from sexual abuse and exploitation. The Act defines various sexual oAenses against children,

including penetrative and non-penetrative sexual assault, sexual harassment, and children's use for

pornographic purposes. The Act provides stringent punishment for those guilty of committing these

Q: Sexual abuse

A: The Protection of Children from Sexual Offences (POCSO) Act, 2012, is an Indian law that protects

children from sexual abuse and exploitation. The Act defines various sexual oAenses against children,

including penetrative and non-penetrative sexual assault, sexual harassment, and children's use for

pornographic purposes. The Act provides stringent punishment for those guilty of co

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


No, it isn't. The Child Labour (Prohibition and Regulation) Act 1986 prohibits the employment of children under 14 in certain hazardous occupations and processes. It regulates the working conditions of children in other occupations. It provides a penalty for employers who violate the Act's provisions and empowers the government to inspect any premises to enforce the Act's provisions. It has been amended several times.



KeyboardInterrupt: Interrupted by user

## References

- [Skyscanner help center](https://help.skyscanner.net/hc/en-us/categories/200385602-Traveling)
- [Embeddings Leaderboard](https://huggingface.co/spaces/mteb/leaderboard)
- [Local LLM Comparison](https://github.com/Troyanovsky/Local-LLM-comparison)
- [Nous-Hermes-13B GPTQ](https://huggingface.co/TheBloke/Nous-Hermes-13B-GPTQ)
- [Nous Hermes 13B (Original model)](https://huggingface.co/NousResearch/Nous-Hermes-13b)
- [Text Embeddings Leaderboard](https://huggingface.co/spaces/mteb/leaderboard)
- [OpenLLM Leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard)